In [23]:
import os
import shutil
import random
from PIL import Image

input_folder = 'Brain Tumor Segmentation'
images_folder = os.path.join(input_folder, 'images')
masks_folder = os.path.join(input_folder, 'masks')


train_images_folder = os.path.join(input_folder, 'train', 'images')
train_masks_folder = os.path.join(input_folder, 'train', 'masks')
test_images_folder = os.path.join(input_folder, 'test', 'images')
test_masks_folder = os.path.join(input_folder, 'test', 'masks')
validation_images_folder = os.path.join(input_folder, 'validation', 'images')
validation_masks_folder = os.path.join(input_folder, 'validation', 'masks')

os.makedirs(train_images_folder, exist_ok=True)
os.makedirs(train_masks_folder, exist_ok=True)
os.makedirs(test_images_folder, exist_ok=True)
os.makedirs(test_masks_folder, exist_ok=True)
os.makedirs(validation_images_folder, exist_ok=True)
os.makedirs(validation_masks_folder, exist_ok=True)

image_files = [f for f in os.listdir(images_folder) if os.path.isfile(os.path.join(images_folder, f))]
random.shuffle(image_files)

split_index = int(len(image_files) * 0.7)
train_images = image_files[:split_index]
remaining_images = image_files[split_index:]
split_index2 = int(len(remaining_images) * 0.5)
test_images = remaining_images[:split_index2]
validation_images = remaining_images[split_index2:]

def resize_and_save(image_path, save_path, size=(128, 128)):
    with Image.open(image_path) as img:
        img = img.resize(size, Image.LANCZOS)
        img.save(save_path)

for img in train_images:
    resize_and_save(os.path.join(images_folder, img), os.path.join(train_images_folder, img))
    resize_and_save(os.path.join(masks_folder, img), os.path.join(train_masks_folder, img))

for img in test_images:
    resize_and_save(os.path.join(images_folder, img), os.path.join(test_images_folder, img))
    resize_and_save(os.path.join(masks_folder, img), os.path.join(test_masks_folder, img))

for img in validation_images:
    resize_and_save(os.path.join(images_folder, img), os.path.join(validation_images_folder, img))
    resize_and_save(os.path.join(masks_folder, img), os.path.join(validation_masks_folder, img))

print(f"Total images: {len(image_files)}")
print(f"Training images: {len(train_images)}")
print(f"Testing images: {len(test_images)}")
print(f"Validation images: {len(validation_images)}")
print("Images and masks successfully split into training and testing folders.")

Total images: 3064
Training images: 2144
Testing images: 460
Validation images: 460
Images and masks successfully split into training and testing folders.


In [ ]:
import numpy as np
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Input, Cropping2D, Flatten
from keras.models import Model
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

# Cuda
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("GPU detected")
    except:
        print("GPU not detected")
        pass
else :
    print("GPU not detected") 

def unet_model(input_shape=(128, 128, 3)):
    inputs = Input(input_shape)
    
    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Dropout(0.3)(c3)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u4 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c3)
    u4 = Concatenate()([u4, c2])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u4)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u5 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = Concatenate()([u5, c1])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u5)
    c5 = Dropout(0.1)(c5)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)

    model = Model(inputs, outputs)
    return model

# Load images
def load_images(images_folder, masks_folder, target_size=(128, 128)):
    image_files = [f for f in os.listdir(images_folder) if os.path.isfile(os.path.join(images_folder, f))]
    mask_files = [f for f in os.listdir(masks_folder) if os.path.isfile(os.path.join(masks_folder, f))]

    assert len(image_files) == len(mask_files), "Mismatach number of images and masks."

    images = []
    masks = []

    for img_file, mask_file in zip(image_files, mask_files):
        img_path = os.path.join(images_folder, img_file)
        mask_path = os.path.join(masks_folder, mask_file)

        image = load_img(img_path, target_size=target_size)
        mask = load_img(mask_path, target_size=target_size, color_mode="grayscale")

        # Normalize images
        image_array = img_to_array(image) / 255.0  
        mask_array = img_to_array(mask) / 255.0    

        images.append(image_array)
        masks.append(mask_array)

    return np.array(images), np.array(masks)

train_folder = 'Brain Tumor Segmentation/train'
train_images_folder = os.path.join(train_folder, 'images')
train_masks_folder = os.path.join(train_folder, 'masks')

test_folder = 'Brain Tumor Segmentation/test'
test_images_folder = os.path.join(test_folder, 'images')
test_masks_folder = os.path.join(test_folder, 'masks')

train_images, train_masks = load_images(train_images_folder, train_masks_folder)
test_images, test_masks = load_images(test_images_folder, test_masks_folder)

# Compile model
model = unet_model(input_shape=(128, 128, 3))
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_images, train_masks, 
    validation_data=(test_images, test_masks),
    batch_size=32,
    epochs=20,
    verbose=1 
)

Epoch 1/20


In [22]:
import os
import shutil
import random
from PIL import Image

input_folder = 'Face_Dataset (1)'
images_folder = os.path.join(input_folder, 'images')
masks_folder = os.path.join(input_folder, 'masks')

test_images_folder = os.path.join(input_folder, 'test', 'images')
test_masks_folder = os.path.join(input_folder, 'test', 'masks')

os.makedirs(test_images_folder, exist_ok=True)
os.makedirs(test_masks_folder, exist_ok=True)

image_files = [f for f in os.listdir(images_folder) if f.lower().endswith('.jpg')]
random.shuffle(image_files)

test_images = image_files

def resize_and_save(image_path, save_path, size=(256, 256)):
    with Image.open(image_path) as img:
        img = img.resize(size, Image.LANCZOS)
        save_path = os.path.splitext(save_path)[0] + '.png'  # Change extension to .png
        img.save(save_path, 'PNG')

for img in test_images:
    img_name = os.path.splitext(img)[0]
    mask_name = img_name + '.png'
    resize_and_save(os.path.join(images_folder, img), os.path.join(test_images_folder, img_name + '.png'))
    resize_and_save(os.path.join(masks_folder, mask_name), os.path.join(test_masks_folder, mask_name))

print(f"Total images: {len(image_files)}")
print(f"Testing images: {len(test_images)}")
print("Images and masks successfully converted to PNG and saved in the test folders.")

Total images: 77
Testing images: 77
Images and masks successfully converted to PNG and saved in the test folders.
